In [1]:
import json
from typing import List, Dict
from datasets import load_dataset

path = "/Users/charlesoneill/retrieval/data/multi_paper.json"

with open(path, 'r') as f:
    data = json.load(f)

# # Remove any data where the arxiv field is empty list
# data = [{k: v} for k, v in data.items() if v['arxiv']]
# print(len(data))

# def _map_arxiv_to_filenames(self, arxiv_ids: List[str]) -> List[str]:
#         #self.document_ids.append(f"{paper['subfolder']}/{paper['filename']}")
#         ids_list = []
#         for arxiv_id in arxiv_ids:
#             if '.' in arxiv_id:
#                 print(arxiv_id)
#                 year_month = arxiv_id.split('.')[0]
#                 identifier = arxiv_id.split('.')[1].split('_')[0]
#                 ids_list.append(f"{year_month}/{year_month}.{identifier}_arXiv.txt")
#             else:
#                 year_month = arxiv_id[:4]
#                 identifier = arxiv_id[4:]
#                 ids_list.append(f"{year_month}/{year_month}.{identifier}_arXiv")
#         return ids_list

# # Apply this to the data
# for d in data:
#     key_zero = list(d.keys())[0]   
#     arxiv_ids = d[key_zero]['arxiv']
#     d[key_zero]['arxiv'] = _map_arxiv_to_filenames(None, arxiv_ids)

# # Save the updated data
# with open(path, 'w') as f:
#     json.dump(data, f)
    

# # Load the huggingface dataset
repo_id = "charlieoneill/jsalt-astroph-dataset"
dataset = load_dataset(repo_id, split="train")

# Get all the filenames from the dataset
filenames = dataset['filename']

# # Only keep citations in the data that are in filenames
# for d in data:
#     key_zero = list(d.keys())[0]
#     arxiv_ids = d[key_zero]['arxiv']
#     d[key_zero]['arxiv'] = [arxiv_id for arxiv_id in arxiv_ids if arxiv_id in filenames]

# # Print how many citations for each d in data
# for d in data:
#     key_zero = list(d.keys())[0]
#     print(len(d[key_zero]['arxiv']))

/Users/charlesoneill/miniconda3/envs/jsalt/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
filenames

['1607.01526_arXiv.txt',
 '1607.03009_arXiv.txt',
 '1607.06240_arXiv.txt',
 '1607.07246_arXiv.txt',
 '1607.07888_arXiv.txt',
 '1607.08820_arXiv.txt',
 '1607.06710_arXiv.txt',
 '1607.08619.txt',
 '1607.08157_arXiv.txt',
 '1607.01847_arXiv.txt',
 '1607.07699_arXiv.txt',
 '1607.02243_arXiv.txt',
 '1607.02596_arXiv.txt',
 '1607.00841_arXiv.txt',
 '1607.05713_arXiv.txt',
 '1607.07534_arXiv.txt',
 '1607.07969_arXiv.txt',
 '1607.06368_arXiv.txt',
 '1607.01062_arXiv.txt',
 '1607.08005_arXiv.txt',
 '1607.06829_arXiv.txt',
 '1607.02707_arXiv.txt',
 '1607.00228_arXiv.txt',
 '1607.02477_arXiv.txt',
 '1607.03701_arXiv.txt',
 '1607.03537_arXiv.txt',
 '1607.01829_arXiv.txt',
 '1607.05726_arXiv.txt',
 '1607.02094.txt',
 '1607.07723_arXiv.txt',
 '1607.05383_arXiv.txt',
 '1607.00874_arXiv.txt',
 '1607.08162_arXiv.txt',
 '1607.04385_arXiv.txt',
 '1607.03100_arXiv.txt',
 '1607.01105_arXiv.txt',
 '1607.08738_arXiv.txt',
 '1607.05795_arXiv.txt',
 '1607.00386_arXiv.txt',
 '1607.07874_arXiv.txt',
 '1607.05040

In [3]:
# Made an Arxiv ID dataclass
from dataclasses import dataclass

@dataclass
class ArxivID:
    year_month: str
    identifier: str

    def __str__(self):
        return f"{self.year_month}.{self.identifier}"

    def __repr__(self):
        return f"{self.year_month}.{self.identifier}"
    
    def __eq__(self, other):
        return self.year_month == other.year_month and self.identifier == other.identifier
    
    def __hash__(self):
        return hash((self.year_month, self.identifier))

In [20]:
# Create a list of Arxiv IDS

def string_to_arxiv_id(arxiv_id: str) -> ArxivID:
    try:
        if 'arXiv' in arxiv_id:
            year_month = arxiv_id.split('.')[0]
            identifier = arxiv_id.split('.')[1].split('_')[0]
        # Elif there are two full stops in the arxiv_id
        elif arxiv_id.count('.') == 2:
            year_month = arxiv_id.split('.')[0]
            identifier = arxiv_id.split('.txt')[0].split('.')[1]
        else:
            # Extract any characters that are numbers
            nums = ''.join([char for char in arxiv_id if char.isdigit()])
            year_month = nums[:4]
            identifier = nums[4:]
    except Exception as e:
        print(e)
        print(arxiv_id)
    return ArxivID(year_month, identifier)

arxiv_ids = [string_to_arxiv_id(arxiv_id) for arxiv_id in filenames]

In [22]:
# Add arxiv_ids as a column to the dataset
dataset = dataset.add_column("arxiv_id", [str(x) for x in arxiv_ids])

In [23]:
# Push to hub
dataset.push_to_hub(repo_id)

Uploading the dataset shards: 100%|██████████| 7/7 [04:26<00:00, 38.10s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/charlieoneill/jsalt-astroph-dataset/commit/bdf91ee8d0bbdde703a1b10070afa19bd4c830d9', commit_message='Upload dataset', commit_description='', oid='bdf91ee8d0bbdde703a1b10070afa19bd4c830d9', pr_url=None, pr_revision=None, pr_num=None)

In [10]:
arxiv_ids

[1607.01526,
 1607.03009,
 1607.06240,
 1607.07246,
 1607.07888,
 1607.08820,
 1607.06710,
 1607.08619,
 1607.08157,
 1607.01847,
 1607.07699,
 1607.02243,
 1607.02596,
 1607.00841,
 1607.05713,
 1607.07534,
 1607.07969,
 1607.06368,
 1607.01062,
 1607.08005,
 1607.06829,
 1607.02707,
 1607.00228,
 1607.02477,
 1607.03701,
 1607.03537,
 1607.01829,
 1607.05726,
 1607.02094,
 1607.07723,
 1607.05383,
 1607.00874,
 1607.08162,
 1607.04385,
 1607.03100,
 1607.01105,
 1607.08738,
 1607.05795,
 1607.00386,
 1607.07874,
 1607.05040,
 1607.05089,
 1607.03982,
 1607.06105,
 1607.08610,
 1607.01677,
 1607.01394,
 1607.04264,
 1607.02442,
 1607.05324,
 1607.03114,
 1607.06394,
 1607.01782,
 1607.06488,
 1607.05442,
 1607.02054,
 1607.07447,
 1607.02781,
 1607.03567,
 1607.05732,
 1607.02975,
 1607.02144,
 1607.08250,
 1607.07918,
 1607.03973,
 1607.06621,
 1607.04412,
 1607.07377,
 1607.03554,
 1607.03886,
 1607.03909,
 1607.03412,
 1607.01403,
 1607.04226,
 1607.06479,
 1607.03540,
 1607.03149,

In [24]:
citations = []

for d in data:
    key_zero = list(d.keys())[0]
    arxiv_ids_citations = d[key_zero]['arxiv']
    arxiv_ids_citations = [x.split('/')[1].split('_')[0] for x in arxiv_ids_citations]
    arxiv_ids_citations = [string_to_arxiv_id(arxiv_id) for arxiv_id in arxiv_ids_citations]
    print(arxiv_ids_citations)
    print()
    # Replace d's arxiv_ids with the ArxivID objects
    d[key_zero]['arxiv'] = [str(x) for x in arxiv_ids_citations]

    # Extend
    citations.extend(arxiv_ids_citations)

[1201.0755, 1212.4819, 1208.3529, 0309.070, 0302.212, 1207.2727, 1212.0860, 1211.6804, 1212.5222, 1301.1228, 0707.2080, 1211.3663, 1210.8417, 1008.3901]

[0810.2788, 0411.364, 1106.1994, 1212.4819, 1208.3529, 0411.391, 0411.424, 1109.1005, 0609.016, 1201.0755]

[0311.060, 0701.924, 1202.0290, 9910.104, 0411.775, 0705.2831, 0312.474, 1011.6370, 1108.0933, 0703.653, 0005.120]

[1602.06956, 1405.2921, 1607.08616, 0904.1995, 1412.2712, 1503.00755, 1407.7040, 1410.8214, 1606.09252, 1309.5951, 1603.05984, 1206.1859, 1508.00007, 1511.00687, 1212.2965]

[0812.4231, 1112.1698, 1205.5037, 1211.5140, 0504.558, 1003.3273, 0604.091, 1205.0270, 0207.562, 1206.1859, 1202.5973, 1511.00687]

[0802.0496, 0106.018, 1310.2253, 0406.632, 1606.06734, 1403.0947, 1111.3980, 0902.4525, 1511.05144]

[1302.1613, 1308.0593, 1303.6430, 0110.368, 0203.375, 0704.1614]

[0004.157, 1412.5649, 0210.628, 0008.075, 1503.02110, 0609.523, 1309.2631, 1409.3566, 0510.313, 0211.551]

[1301.3168, 1309.0809, 1301.5318, 0203.375

In [26]:
data

[{'1403.0007_1': {'title': 'Cosmic Star-Formation History',
   'question': ' How has the UV luminosity function of galaxies evolved from z~4 to z~8 according to analyses of deep HST data?',
   'text': 'Early analyses of the GOODS data (Giavalisco et al. 2004a) found evidence for relatively mild evolution of the UV luminosity density from 2 < z < 5 and clear evidence that there were fewer high-luminosity galaxies at z≈6 (Stanway et al. 2003, Dickinson et al. 2004). Subsequent studies have repeatedly mined the combined GOODS+HUDF observations, using deeper data and more rigorous analyses. At least 20 papers about high-redshift LFs using WFC3 data in the HUDF and GOODS/CANDELS have been published since 2010, augmenting at least a dozen others pre-WFC3. We cannot attempt to review them all here, but they have convincingly demonstrated that the UVLF evolves significantly at z>4. The current consensus is that this is primarily luminosity evolution, at least at 4 < z < 7, with L∗ brightening 

In [27]:
# Save the updated data
with open(path, 'w') as f:
    json.dump(data, f)

In [18]:
citations

[1201.0755,
 1212.4819,
 1208.3529,
 0309.070,
 0302.212,
 1207.2727,
 1212.0860,
 1211.6804,
 1212.5222,
 1301.1228,
 0707.2080,
 1211.3663,
 1210.8417,
 1008.3901,
 0810.2788,
 0411.364,
 1106.1994,
 1212.4819,
 1208.3529,
 0411.391,
 0411.424,
 1109.1005,
 0609.016,
 1201.0755,
 0311.060,
 0701.924,
 1202.0290,
 9910.104,
 0411.775,
 0705.2831,
 0312.474,
 1011.6370,
 1108.0933,
 0703.653,
 0005.120,
 1602.06956,
 1405.2921,
 1607.08616,
 0904.1995,
 1412.2712,
 1503.00755,
 1407.7040,
 1410.8214,
 1606.09252,
 1309.5951,
 1603.05984,
 1206.1859,
 1508.00007,
 1511.00687,
 1212.2965,
 0812.4231,
 1112.1698,
 1205.5037,
 1211.5140,
 0504.558,
 1003.3273,
 0604.091,
 1205.0270,
 0207.562,
 1206.1859,
 1202.5973,
 1511.00687,
 0802.0496,
 0106.018,
 1310.2253,
 0406.632,
 1606.06734,
 1403.0947,
 1111.3980,
 0902.4525,
 1511.05144,
 1302.1613,
 1308.0593,
 1303.6430,
 0110.368,
 0203.375,
 0704.1614,
 0004.157,
 1412.5649,
 0210.628,
 0008.075,
 1503.02110,
 0609.523,
 1309.2631,
 1409

In [19]:
# Determine how many of the elements in citations are in arxiv_ids
citations = set(citations)
arxiv_ids = set(arxiv_ids)

len(citations.intersection(arxiv_ids))

1288

In [34]:
# Do the same for single_paper.json
path = "/Users/charlesoneill/retrieval/data/single_paper.json"

with open(path, 'r') as f:
    data = json.load(f)

new_data = {}

# But now the keys are what we need to iterate over
for k, v in data.items():
    new_key = k.split('/')[1].split('_')[0]
    new_data[new_key] = v

print(new_data)

{'1006.1889': {'question_intro': 'How do convective instabilities in core collapse supernovae contribute to the modulation of neutrino emission and what potential implications could the detection of such variations have on our understanding of supernova mechanisms?', 'question_conclusion': 'How do the time-varying neutrino signals from a galactic supernova depend on the direction of viewing the supernova and the distance from the observer?'}, '1009.2001': {'question_intro': 'What is the evidence for magnetic activity in evolved M giants and how does it compare to other late-type stars in terms of magnetic field measurements?', 'question_conclusion': 'What mechanisms could potentially drive dynamo operation in M giant stars and how can we differentiate between different types of dynamos based on observational data?'}, '1010.3635': {'question_intro': 'How do magnetic fields influence the formation and shaping of protostellar systems in both low-mass and high-mass star-forming regions?', 

In [35]:
# Save the updated data
with open(path, 'w') as f:
    json.dump(new_data, f)

In [40]:
# Load the multi_paper dataset
path = "/Users/charlesoneill/retrieval/data/multi_paper.json"

with open(path, 'r') as f:
    data = json.load(f)

data

{'1403.0007': {'title': 'Cosmic Star-Formation History',
  'question': ' How does the specific star formation rate of galaxies evolve with redshift and stellar mass, and what are the implications for galaxy growth and the cosmic star formation history?',
  'text': 'In recent years, there has been considerable interest in the sSFR (sSFR≡SFR/M∗) of galaxies with different masses at different times in the history of the Universe. The sSFR describes the fractional growth rate of stellar mass in a galaxy or, equivalently, the ratio of current to past star formation. The inverse of the sSFR is the characteristic stellar mass doubling time (Guzman et al. 1997). At 0<z<2 and quite possibly at higher redshifts, most star-forming galaxies follow a reasonably tight relation between SFR and M∗, whose normalization (e.g., the mean sSFR at some fiducial mass) decreases steadily with cosmic time (decreasing redshift) at least from z = 2 to the present (Brinchmann et al. 2004, Daddi et al. 2007, Elbaz

In [38]:
# Load the multi_paper dataset
path = "/Users/charlesoneill/retrieval/data/multi_paper.json"

with open(path, 'r') as f:
    data = json.load(f)

# Remove everything after and including the '_' in each key
new_data = {}

for d in data:
    key_zero = list(d.keys())[0]
    new_key = key_zero.split('_')[0]
    new_data[new_key] = d[key_zero]

new_data

{'1403.0007': {'title': 'Cosmic Star-Formation History',
  'question': ' How does the specific star formation rate of galaxies evolve with redshift and stellar mass, and what are the implications for galaxy growth and the cosmic star formation history?',
  'text': 'In recent years, there has been considerable interest in the sSFR (sSFR≡SFR/M∗) of galaxies with different masses at different times in the history of the Universe. The sSFR describes the fractional growth rate of stellar mass in a galaxy or, equivalently, the ratio of current to past star formation. The inverse of the sSFR is the characteristic stellar mass doubling time (Guzman et al. 1997). At 0<z<2 and quite possibly at higher redshifts, most star-forming galaxies follow a reasonably tight relation between SFR and M∗, whose normalization (e.g., the mean sSFR at some fiducial mass) decreases steadily with cosmic time (decreasing redshift) at least from z = 2 to the present (Brinchmann et al. 2004, Daddi et al. 2007, Elbaz

In [39]:
# Save to multipaper again
with open(path, 'w') as f:
    json.dump(new_data, f)